In [1]:
from lsst.daf.persistence import Butler
import lsst.qa.explorer
import lsst.qa.explorer.functors
from importlib import reload

In [2]:
import lsst.ap.association as assoc

In [3]:
b = Butler("/project/mrawls/hits2015/rerun/comparewarp3")

In [34]:
data = b.get("deepDiff_diaSrc", visit=410915, ccdnum=25)
diffIm = b.get("deepDiff_differenceExp", visit=410915, ccdnum=25)
calib = diffIm.getCalib()

In [ ]:
data.schema

In [5]:
dpddifier = assoc.MapDiaSourceTask(data.schema)

In [6]:
dia_sources = dpddifier.run(data, b.get("deepDiff_differenceExp", visit=410915, ccdnum=25))

In [7]:
diaSrcAstro = data.asAstropy().to_pandas()
p = lsst.qa.explorer.parquetTable.ParquetTable(dataFrame=diaSrcAstro)

In [38]:
funcs = lsst.qa.explorer.functors.CompositeFunctor.from_file(
    '/home/morriscb/src/ap_association/data/diaSourceSDM.yaml')
for name, func in funcs.funcDict.items():
    if isinstance(func, lsst.qa.explorer.functors.NanoJansky):
        funcs.funcDict[name] = lsst.qa.explorer.functors.NanoJansky(colFlux=func.col, colFluxEr=func.colFluxErr,
                                                                    calib=calib)
    elif isinstance(func, lsst.qa.explorer.functors.NanoJanskyErr):
        funcs.funcDict[name] = lsst.qa.explorer.functors.NanoJanskyErr(colFlux=func.col, colFluxErr=func.colFluxErr,
                                                                       calib=calib)

diaSourceId {'functor': 'Column', 'args': 'id'}
ra {'functor': 'RAColumn'}
decl {'functor': 'DecColumn'}
ra_decl_Cov {'functor': 'DecColumn'}
x {'functor': 'Column', 'args': 'slot_Centroid_x'}
xErr {'functor': 'Column', 'args': 'slot_Centroid_xErr'}
y {'functor': 'Column', 'args': 'slot_Centroid_y'}
yErr {'functor': 'Column', 'args': 'slot_Centroid_yErr'}
xy_flag {'functor': 'Column', 'args': 'slot_Centroid_flag'}
apFlux {'functor': 'NanoJansky', 'args': 'slot_ApFlux_instFlux'}
apFluxErr {'functor': 'NanoJanskyErr', 'args': ['slot_ApFlux_instFlux', 'slot_ApFlux_instFluxErr']}
psFlux {'functor': 'NanoJansky', 'args': 'slot_PsfFlux_instFlux'}
psFluxErr {'functor': 'NanoJanskyErr', 'args': ['slot_PsfFlux_instFlux', 'slot_PsfFlux_instFluxErr']}
totFlux {'functor': 'NanoJansky', 'args': 'ip_diffim_forced_PsfFlux_instFlux'}
totFluxErr {'functor': 'NanoJanskyErr', 'args': ['ip_diffim_forced_PsfFlux_instFlux', 'ip_diffim_forced_PsfFlux_instFluxErr']}
ixx {'functor': 'Column', 'args': 'slot_Sha

In [39]:
result = funcs(p, dropna=False)

In [40]:
funcs.funcDict['psFlux'].fluxMag0Err

181778618.958785

In [45]:
result[['psFlux', 'psFluxErr']]

,psFlux,psFluxErr
0,3.586196e+03,420.102784
1,-1.922052e+03,382.790341
2,2.673523e+06,1935.716232
3,1.917447e+03,346.547356
4,-4.679925e+02,346.127025
5,-8.039224e+03,364.389990
6,-6.277467e+03,281.987441
7,-2.404431e+03,279.278923
8,-3.423526e+03,398.500893
9,NaN,NaN


In [ ]:
nJ(p, dropna=False)

In [48]:
dia_sources['psFluxErr']

array([ 420.102784  ,  382.79034084, 1935.71623234,  346.5473559 ,
        346.12702497,  364.38999022,  281.98744114,  279.27892324,
        398.50089258,           nan,  316.08868737,  583.25267012,
        322.05402679,  346.68080104,  347.12488424, 2161.38245245,
       2295.44896998,  346.06807921,  342.54014728,  317.78623072,
        341.54841625,  273.06222149,  422.98179614,  357.64443737,
                 nan,  343.72607132,  326.53469383,  345.48924715,
        341.95222461,           nan,           nan,  300.8876192 ,
        345.12023603,  345.33658985,  275.53503127,  347.41293914,
        345.46552775,           nan, 2526.99474409,  346.97022108,
        344.31762794,           nan,  344.99295837,           nan,
       2815.5144791 ,  346.18814117,  343.14219704,  345.10782166,
                 nan,  348.20484939,  349.13839587,  330.67380607,
       2462.46123232,  347.30447374,  500.59427228,           nan,
        310.86506026,  346.78429717,           nan,  348.03084